In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt #usamos pontualmente para imprimir um Elbow Chart
from kmodes.kmodes import KModes #usamos para classificar nossos dados grupos de acordo com a Moda
import scipy.stats as stats # usamos para verificar correlações (estatística e a significancia) 

ModuleNotFoundError: No module named 'kmodes'

## Divisão em Clusters usando Modas (K-modes):
à procura de grupos/categorias não semelhantes entre si

Nem sempre é possível MANUALMENTE identificar grupos numa análise descritiva. Usaremos o K-modes para CLASSIFICAR os dados em grupos não semelhantes

In [ ]:
# Separando as variáveis por Categórias para facilitar o entendimento

variaveis_nominais = ['Gênero', 'Raça', 'Formação', 'Departamento', 'Estado']
variaveis_ordinais = ['Senioridade']
variaveis_continuas = ['Idade', 'Tempo de Empresa']

lista = variaveis_nominais + variaveis_ordinais

In [ ]:
# Usando dict comprehention para verificar os valores unicos em cada coluna antes de começarmos

dict = {i: df[i].unique() for i in lista}

dict

In [ ]:
#Transformando todas as colunas em strings pois só assim podemos usar o pd.get_dummies [função que transforma as variáveis em dummies]

df_so_categoricas = df[lista]

for col in df_so_categoricas.columns:
    df_so_categoricas.loc[:,col] = df_so_categoricas[col].astype(str)

df_dummy = pd.get_dummies(df_so_categoricas, drop_first=False)
df_dummy.head()

In [ ]:
# Curva 'Elbow' (cotovelo) para encontrar o K ótimo [K = número de clusters]

custo = []
K = range(1,5)
for num_clusters in K:
    kmode = KModes(n_clusters=num_clusters, init = "random", n_init = 5, verbose=1)
    kmode.fit_predict(df)
    custo.append(kmode.cost_)
    
plt.plot(K, custo, 'bx-')
plt.xlabel('No. de Clusters')
plt.ylabel('Custo')
plt.title('Gráfico Elbow para Encontrar o K Otimo')
plt.show()

In [ ]:
# Na Curva Elbow o K ótimo encontrado foi de 2. Assim definimos o número de clusters ideal como 2
num_clusters = 2

# Criando uma instância do modelo k-modes
km = KModes(n_clusters=num_clusters, init='Huang', n_init=5, verbose=1)

# Ajustando o modelo
clusters = km.fit_predict(df)

# Adicionando os resultados dos clusters ao DataFrame original
df_dummy['Cluster'] = clusters
df_cluster = df_dummy

df_cluster.head()

In [ ]:
# Verificando quantas pessoas possue cada cluster
display(df_cluster['Cluster'].value_counts())

# Verificando sua proporção
df_cluster['Cluster'].value_counts(normalize=True).apply(lambda x: str(round(x*100, 2)) + "%")

In [ ]:
# Imprimindo a porcentagem
var_temporaria = df_cluster.groupby('Cluster').mean()
var_temporaria

# Exportando para uma visualização mais palatável
# var_temporaria.to_csv('.\Cluster.csv')

![Descrição da imagem](image1.jpeg)

Conclusões :

Existem 2 grupos com características marcantes

Grupo 1 - Majoritariamente composto por Homens, Brancos, com Formação Superior, concentrados no Departamento Administrativo e que em sua maioria trabalham como Analistas Plenos

Grupo 2 - Majoritariamente composto por Homens, Pardos, com Formação Superior, concetrados principalmente no Departamento de TI e que em sua maioria trabalham como Analistas Júnior


A partir destas distinções, podemos ter uma noção mais clara e CONJUNTA de quais áreas da nossa empresa carecem de mais diversidade e inclusão

## Inferências a respeito do "tempo de empresa" : 
à procura das "causas" do nosso alto Churn Rate visando a criação de uma política de retenção de colaboradores eficiente

In [ ]:
df.head()

In [ ]:
df_dummy.columns

In [ ]:
dict_corr = {'Coluna': [], 'Correlação': [], 'P-Value': []}

# Estamos aplicando uma correlação chamada ponto bisserial que é uma variação da correlação de person aplicada para sabermos a relação entre uma variável categórica e uma contínua

for column in df_dummy.columns:
    corr, pvalue = stats.pointbiserialr(df_dummy[column], df['Tempo de Empresa'])
    dict_corr['Coluna'].append(column)
    dict_corr['Correlação'].append(round(corr, 4))
    dict_corr['P-Value'].append(round(pvalue, 4))

df_corr = pd.DataFrame(dict_corr)

df_corr_filtrado_1 = df_corr.loc[df_corr['Correlação'] > 0.02]
display(df_corr_filtrado_1)

df_corr_filtrado_2 = df_corr.loc[df_corr['P-Value'] < 0.05]
df_corr_filtrado_2

Conclusões : 

De forma surpreendente, não existe NENHUMA correlação na nossa base de dados que aponte para a retenção de de colaborades de forma significativa